In [1]:
import geopandas as gp
from shapely import wkt
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
import pandas as pd
import numpy as np
from pprint import pprint
import os
import glob
import openpyxl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import plotly.express as px #if using plotly
import folium
import warnings
import random

In [2]:
pd.set_option('display.max_columns', None)  # display all columns
pd.options.display.float_format = '{:20,.2f}'.format  # suppress scientific notation

## Research Questions

### 1) Is there a significant difference in the proportion of minorities within a given flaring buffer zone versus outside the buffer? 

#### Ultimate goal is a comparison of six different buffers (of 100; 400; 800; 1,000; 1,600; and 2,000 m) following Czolowski et al, 2017

Null: Proportion of BG in a buffer is independent of minority status.  
Alt:  Proportion of BG in a buffer is related to minority status.
  
Test stat: Difference in weighted minority proportions.

Weighted minority proportion = sumproduct(minority_prop * intersect_prop) / sum(intersect_prop)
minority_prop is the proportion of minorities in a block group
intersect_prop is the proportion of BG area residing in the buffer zone

Permutation testing: Permute the intersect_prop variable, holding the fraction of minorities in each BG constant.
 
Process   
a) Calculate the actual minority proportions of the aggregrate in-buffer and out-buffer areas
Using 2km buffer for starters (Czolowski, 2017)  
b) Scramble the intersect_prop variable, holding the minority_prop variable constant.   
c) Calculate number of simulated proportions that match or exceed the actual proportion. Calculate p-value.  


## Load data, set project CRS

In [3]:
ca_state = gp.read_file("data/CA_State_TIGER2016.shp")  # CA state
ca_counties = gp.read_file("data/CA_Counties_TIGER2016.shp")  # CA counties
ca_bg = gp.read_file("data/ca_bg_clean.shp")  # CA block groups, cleaned

In [4]:
# just CA flares
ca_flares = gp.read_file("data/ca_flares_clean.shp")
print(f'Flares found: {len(ca_flares)}')

# CA block groups merged with EJscreen data
ca_bg_joined = gp.read_file("data/ca_bg_joined_clean.shp", dtype={'bg_id': 'int', 'index':'int'})

# update col name for correct area b/c it gets saved as a truncated version when saved as a shapefile out of the 
# main data cleaning notebook
ca_bg_joined.rename(columns={'shape_ar_1':'shape_area_new'}, inplace=True)  # old:new.
print(f'BGs found: {len(ca_bg_joined)}')

Flares found: 117
BGs found: 25607


In [5]:
# set common crs for project
# epsg3310: https://epsg.io/3310-1739
# units: meters
meters_crs = 3310  # Projected crs. this should be good for this overlay() calculation and all of project. 

ca_state = ca_state.to_crs(meters_crs)
ca_counties = ca_counties.to_crs(meters_crs)
ca_bg = ca_bg.to_crs(meters_crs)

#### Discuss w RMI: dropping the three aquatic buffers

In [6]:
# There are three BGs that seem to just be aquatic buffers around the actual county land.
# Dropping them for now 
# IDs: 60839900000, 61119901000, 60379902000
# Explore if needed: ca_bg_joined.explore()

ids_to_drop = [60839900000, 61119901000, 60379902000]

# Drop the rows with those IDs
ca_bg_joined = ca_bg_joined[~ca_bg_joined['bg_id'].isin(ids_to_drop)]

### Break things back out of functions, troubleshoot step by step to find issue with the out_buffer calculation coming out to ~0.60 for all buffer sizes. 

In [7]:
ca_counties.rename(columns={'NAMELSAD':'CNTY_NAME'}, inplace=True)  # old:new. Match col names for merging

ca_bg_joined_sub = ca_bg_joined[['bg_id', 'CNTY_NAME', 'ACSTOTPOP', 'MINORPOP',
                           'D_PM25_2', 'shape_area_new', 'geometry']].copy()

ca_bg_joined_sub.rename(columns={'geometry':'bg_geom'}, inplace=True)  # old:new. Match col names for merging

counties_sub = ca_counties[['CNTY_NAME', 'geometry']].copy()

counties_sub.rename(columns={'geometry':'cnty_geom'}, inplace=True)  # old:new. Match col names for merging


In [8]:
na_counts = ca_bg_joined_sub.isna().sum()


In [9]:
na_counts

bg_id              0
CNTY_NAME          0
ACSTOTPOP          0
MINORPOP           0
D_PM25_2          14
shape_area_new     0
bg_geom            0
dtype: int64

In [10]:
# add county names to social dataframe

ca_bg_joined_sub = pd.merge(ca_bg_joined_sub, counties_sub, 
                             on='CNTY_NAME', how='left')

In [11]:
ca_bg_joined_sub.sample(2)

,bg_id,CNTY_NAME,ACSTOTPOP,MINORPOP,D_PM25_2,shape_area_new,bg_geom,cnty_geom
20321,60971532012,Sonoma County,1559,1291,50.38,"1,601,142.13","POLYGON ((-238046.017 46281.522, -238045.053 4...","POLYGON ((-256278.644 37049.105, -256281.607 3..."
21148,60375020051,Los Angeles County,1667,1492,52.85,"276,230.14","POLYGON ((181627.607 -450009.499, 181638.265 -...","MULTIPOLYGON (((123902.356 -503236.489, 123833..."


In [12]:
def create_buffer_intersection(flares_df, social_df, buffer_size):
    #social_df = social_df[['bg_id', 'CNTY_NAME', 'ACSTOTPOP', 'MINORPOP',
    #                       'D_PM25_2', 'shape_area_new', 'bg_geom', 'cnty_geom']]
    
    social_df = social_df.set_geometry('bg_geom')
    
    #flares_df = set_geometry_buffer(flares_df, buffer_size)
    buffer_col = f"buffer_{buffer_size}m"
    flares_df[buffer_col] = flares_df['geometry'].buffer(distance=buffer_size)

    flares_df = flares_df.set_geometry(buffer_col)
    
    temp = flares_df.unary_union
    all_buffers = gp.GeoDataFrame({'geometry': [temp]}, crs=flares_df.crs)  # convert back to geodf for processing
    
    intersect_temp = gp.overlay(social_df, all_buffers, how='intersection')  # could look at keep_geom=False
    
    intersect_temp.rename(columns={'geometry':'intersect_geom'}, inplace=True)  # old:new. Match col names for merging
    
    return intersect_temp

In [13]:
buffer_sizes = [1000]#, 400, 800, 1000, 1600, 2000]#, 50000, 100000]

for buffer_size in buffer_sizes:
    intersect_df = create_buffer_intersection(ca_flares, ca_bg_joined_sub, buffer_size)

In [14]:
# ensure that the intersections, not the counties, are set as the geometry for calculations.
intersect_df = intersect_df.set_geometry('intersect_geom')  # set to the buffers rather than the points

# Create new 'area' column for the areas of the intersections
intersect_df['intersect_area'] = intersect_df.area

# Calculate the proportion of each block group within the buffer zone
intersect_df['intersect_prop'] = intersect_df['intersect_area'] / intersect_df['shape_area_new']

# subset full social df down to just the rows that are in the intersected counties
county_names = intersect_df['CNTY_NAME'].unique()
social_counties = ca_bg_joined_sub[ca_bg_joined_sub['CNTY_NAME'].isin(county_names)]

# merge the intersections data back into the (subsetted) social df. This creates a df of all BGs — those
# that contain intersections and those that do not — in the counties that have an intersection. 

# I'm creating a subset of California, in effect, to combat against the extremely skewed dataset when I 
# compare the in/out buffer proportions across all of Cali. 
bgs_intersections = pd.merge(social_counties, 
                             intersect_df[['bg_id','intersect_geom','intersect_area', 'intersect_prop']], 
                             on=['bg_id'], 
                             how='left').fillna({'intersect_area': 0, 'intersect_prop': 0})


# Apply the proportion to each demographic variable to find counts by variable
demo_vars = ['ACSTOTPOP', 'MINORPOP']
for var in demo_vars:
    bgs_intersections[var + '_intersect_count'] = bgs_intersections[var] * bgs_intersections['intersect_prop']

# find overall proportions for each demo var by dividing var count by respective total population    
for var in demo_vars:
    bgs_intersections[var + '_bg_totprop'] = np.where(bgs_intersections['ACSTOTPOP'] != 0, 
                                                      bgs_intersections[var] / bgs_intersections['ACSTOTPOP'], 0)

bgs_intersections['outintersect_prop'] = 1-bgs_intersections['intersect_prop']

In [15]:
# take repeated samples of BGs in intersected counties and average the in/out props found
random.seed(42)
num_samples = 20
num_bgs = len(intersect_df)  # do a 4:1 ratio of intersected BGs to all others

# Initialize lists to store the statistics for each sample
in_buffer_list = []
out_buffer_list = []

# Loop through the samples
for i in range(num_samples):
    # Sample rows from the bgs_intersections dataframe
    sample_df = bgs_intersections.sample(n=num_bgs)

    # Calculate the in_buffer and out_buffer proportions for the sample
    sampl_in_buffer = (np.sum(sample_df['MINORPOP_bg_totprop'] * sample_df['intersect_prop']) /
                       np.sum(sample_df['intersect_prop'])) #if np.sum(sample_df['intersect_prop']) != 0 else 0)
    sampl_out_buffer = (np.sum(sample_df['MINORPOP_bg_totprop'] * sample_df['outintersect_prop']) /
                        np.sum(sample_df['outintersect_prop'])) #if np.sum(sample_df['outintersect_prop']) != 0 else 0)

    
    # Calculate the in_buffer and out_buffer proportions for the sample
    #sampl_in_buffer = np.sum(sample_df['MINORPOP_bg_totprop'] * sample_df['intersect_prop']) / np.sum(sample_df['intersect_prop'])
   #sampl_out_buffer = np.sum(sample_df['MINORPOP_bg_totprop'] * sample_df['outintersect_prop']) / np.sum(sample_df['outintersect_prop'])

    # Append the statistics to the lists
    in_buffer_list.append(sampl_in_buffer)
    out_buffer_list.append(sampl_out_buffer)

# Calculate the average statistics for all the samples
mean_in_buffer = np.mean(in_buffer_list)
mean_out_buffer = np.mean(out_buffer_list)

    # Calculate the in_buffer and out_buffer proportions for the sample
in_buffer = np.sum(bgs_intersections['MINORPOP_bg_totprop'] * bgs_intersections['intersect_prop']) / np.sum(bgs_intersections['intersect_prop'])
out_buffer = np.sum(bgs_intersections['MINORPOP_bg_totprop'] * bgs_intersections['outintersect_prop']) / np.sum(bgs_intersections['outintersect_prop'])


/Users/natebender/.local/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


In [16]:
print(f" buffer: *{buffer_size}m*")
#print(f" mean intersection proportion*1000: {mean_intersect_prop:.10f}")
#print(f" mean out-intersection proportion: {mean_outintersect_prop:.10f}")
#print(f" mean intersection MINORPOP count: {mean_intersect_MINORcount:.10f}")
print(f" BGs in buffer: {len(intersect_df)} out of total: {len(social_counties)}")
print(f" People in buffer: {round(sum(bgs_intersections['ACSTOTPOP_intersect_count'])):,} out of total: {sum(social_counties['ACSTOTPOP']):,}")
print()
print(f" OLD in_buffer prop: {in_buffer}")
print(f" OLD out_buffer prop: {out_buffer}")
print()
print(f" sampled in_buffer prop: {mean_in_buffer}")
print(f" sampled out_buffer prop: {mean_out_buffer}")
print()
print("in buffer sample props")
print(in_buffer_list)
print()
print("out buffer sample props")
print(out_buffer_list)


 buffer: *1000m*
 BGs in buffer: 90 out of total: 14533
 People in buffer: 22,852 out of total: 22,780,258

 OLD in_buffer prop: 0.5936777332601038
 OLD out_buffer prop: 0.6481100516655275

 sampled in_buffer prop: nan
 sampled out_buffer prop: 0.6512251279115872

in buffer sample props
[nan, nan, nan, nan, 0.42888343503654613, nan, nan, 0.3671274961597542, nan, nan, 0.15535714285714286, 0.7423547196967887, nan, 0.5954500494559841, nan, 0.6872659176029963, 0.792819429778247, nan, nan, 0.87527352297593]

out buffer sample props
[0.6348541892516862, 0.6739296944267719, 0.6820689556154079, 0.6553255346070042, 0.6612058393564925, 0.6601407093776741, 0.6970187229411421, 0.6247557784958692, 0.6306046437179533, 0.6534236417134291, 0.6636816699438356, 0.6512168936049768, 0.6806810361810651, 0.5978329739564857, 0.6708613837967451, 0.6228526112752304, 0.6690226414496789, 0.657149909675341, 0.5727942423788163, 0.6650814864661382]


In [102]:
def calculate_stats(intersect_df, social_df, buffer_size):

    # ensure that the intersections, not the counties, are set as the geometry for calculations.
    intersect_df = intersect_df.set_geometry('intersect_geom')  # set to the buffers rather than the points

    # Create new 'area' column for the areas of the intersections
    intersect_df['intersect_area'] = intersect_df.area

    # Calculate the proportion of each block group within the buffer zone
    intersect_df['intersect_prop'] = intersect_df['intersect_area'] / intersect_df['shape_area_new']

    # subset full social df down to just the rows that are in the intersected counties
    county_names = intersect_df['CNTY_NAME'].unique()
    social_counties = social_df[social_df['CNTY_NAME'].isin(county_names)]
    
    # merge the intersections data back into the (subsetted) social df. This creates a df of all BGs — those
    # that contain intersections and those that do not — in the counties that have an intersection. 

    # I'm creating a subset of California, in effect, to combat against the extremely skewed dataset when I 
    # compare the in/out buffer proportions across all of Cali. 

    bgs_intersections = pd.merge(social_counties, 
                                 intersect_df[['bg_id','intersect_geom','intersect_area', 'intersect_prop']], 
                                 on=['bg_id'], 
                                 how='left').fillna({'intersect_area': 0, 'intersect_prop': 0})


    # Apply the proportion to each demographic variable to find counts by variable
    demo_vars = ['ACSTOTPOP', 'MINORPOP']
    for var in demo_vars:
        bgs_intersections[var + '_intersect_count'] = bgs_intersections[var] * bgs_intersections['intersect_prop']

    # find overall proportions for each demo var by dividing var count by respective total population    
    for var in demo_vars:
        bgs_intersections[var + '_bg_totprop'] = np.where(bgs_intersections['ACSTOTPOP'] != 0, 
                                                          bgs_intersections[var] / bgs_intersections['ACSTOTPOP'], 0)

    bgs_intersections['outintersect_prop'] = 1-bgs_intersections['intersect_prop']
    
    # take repeated samples of BGs in intersected counties and average the in/out props found
    random.seed(42)
    num_samples = 10
    num_bgs = len(intersect_df)*4  # do a 4:1 ratio of intersected BGs to all others

    # Initialize lists to store the statistics for each sample
    in_buffer_list = []
    out_buffer_list = []

    # Loop through the samples
    for i in range(num_samples):
        # Sample rows from the bgs_intersections dataframe
        sample_df = bgs_intersections.sample(n=num_bgs)

        # Calculate the in_buffer and out_buffer proportions for the sample
        sampl_in_buffer = np.sum(sample_df['MINORPOP_bg_totprop'] * sample_df['intersect_prop']) / np.sum(sample_df['intersect_prop'])
        sampl_out_buffer = np.sum(sample_df['MINORPOP_bg_totprop'] * sample_df['outintersect_prop']) / np.sum(sample_df['outintersect_prop'])

        # Append the statistics to the lists
        in_buffer_list.append(sampl_in_buffer)
        out_buffer_list.append(sampl_out_buffer)

    # Calculate the average statistics for all the samples
    mean_in_buffer = np.mean(in_buffer_list)
    mean_out_buffer = np.mean(out_buffer_list)

        # Calculate the in_buffer and out_buffer proportions for the sample
    in_buffer = np.sum(bgs_intersections['MINORPOP_bg_totprop'] * bgs_intersections['intersect_prop']) / np.sum(bgs_intersections['intersect_prop'])
    out_buffer = np.sum(bgs_intersections['MINORPOP_bg_totprop'] * bgs_intersections['outintersect_prop']) / np.sum(bgs_intersections['outintersect_prop'])

    
    #mean_intersect_prop = round(np.mean(bgs_intersections['intersect_prop'])*100,3) # artificially inflate otherwise they're too small
    #mean_outintersect_prop = round(np.mean(bgs_intersections['outintersect_prop']),3) # artificially inflate otherwise they're too small
    #mean_intersect_MINORcount = round(np.mean(bgs_intersections['MINORPOP_intersect_count']),3)
    
    # table that compares the in-buffer proportions to the outside-buffer proportions
    #temp = pd.DataFrame({'In_Buffer': [in_buffer], 'Outside_Buffer': [out_buffer]})
    print(f" buffer: *{buffer_size}m*")
    #print(f" mean intersection proportion*1000: {mean_intersect_prop:.10f}")
    #print(f" mean out-intersection proportion: {mean_outintersect_prop:.10f}")
    #print(f" mean intersection MINORPOP count: {mean_intersect_MINORcount:.10f}")
    print(f" BGs in buffer: {len(intersect_df)} out of total: {len(social_counties)}")
    print(f" People in buffer: {round(sum(bgs_intersections['ACSTOTPOP_intersect_count'])):,} out of total: {sum(social_counties['ACSTOTPOP']):,}")
    print()
    print(f" OLD in_buffer prop: {in_buffer}")
    print(f" OLD out_buffer prop: {out_buffer}")
    print()
    print(f" sampled in_buffer prop: {mean_in_buffer}")
    print(f" sampled out_buffer prop: {mean_out_buffer}")
    print()
    
    
    # drop the geometries before writing to files; they're not needed for permutation
    #bgs_intersections_drop = bgs_intersections.drop(["bg_geom", "intersect_geom", "cnty_geom"], axis=1)    
    #filename = f"data/df_{buffer_size}m_forpermutation.tsv"
    #bgs_intersections_drop.to_csv(filename, sep="\t", index=False)
    
    return


In [103]:
buffer_sizes = [100]#, 400, 800, 1000, 1600, 2000]#, 50000, 100000]

for buffer_size in buffer_sizes:
    intersect = create_buffer_intersection(ca_flares, ca_bg_joined_sub, buffer_size)
    calculate_stats(intersect, ca_bg_joined_sub, buffer_size)



 buffer: *100m*
 BGs in buffer: 38 out of total: 14533
 People in buffer: 117 out of total: 22,780,258

 OLD in_buffer prop: 0.38544330337833693
 OLD out_buffer prop: 0.6480548606808851

 sampled in_buffer prop: nan
 sampled out_buffer prop: 0.6482683627683614



/Users/natebender/.local/lib/python3.7/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
# bgs_intersections = bgs_intersections.set_geometry('intersect_geom')  # set to the buffers rather than the points

# # Filter out invalid geometries
# valid_geometries = bgs_intersections[bgs_intersections.geometry.is_valid]

# # Visualize the valid geometries
# #valid_geometries.explore()

# len(valid_geometries)

In [ ]:
# bgs_intersections = bgs_intersections.set_geometry('bg_geom')  # set to the buffers rather than the points

# # Filter out invalid geometries
# valid_geometries = bgs_intersections[bgs_intersections.geometry.is_valid]

# # Visualize the valid geometries
# #valid_geometries.explore()

# len(valid_geometries)

In [ ]:
# ca_bg_joined_sub.to_csv("data/ca_bg_sub_testing_calc.csv")
# intersect.to_csv("data/intersect_testing_calc.csv")

In [ ]:
# bgs_intersections.sort_values(by='intersect_prop', ascending=False)

In [ ]:

# in_buffer = np.sum(bgs_intersections['MINORPOP_bg_totprop'] * bgs_intersections['intersect_prop']) / np.sum(bgs_intersections['intersect_prop'])
# out_buffer = np.sum(bgs_intersections['MINORPOP_bg_totprop'] * bgs_intersections['outintersect_prop']) / np.sum(bgs_intersections['outintersect_prop'])

# # create a table from the in-buffer and outside-buffer proportions
# # proportions = pd.concat([in_buffer, out_buffer], axis=1)
# # proportions.columns = ['In_Buffer', 'Outside_Buffer']

# # table that compares the in-buffer proportions to the outside-buffer proportions
# temp2000 = pd.DataFrame({'In_Buffer': [in_buffer], 'Outside_Buffer': [out_buffer]})
print(temp2000)

## Analysis
### Functions

In [ ]:
def calc_intersect_stats(bg_inbuffer, social_df, ca_blockgroups, buffer_size):
    
    # Create new 'area' column for the areas of the intersections
    bg_inbuffer['intersect_area'] = bg_inbuffer.area
    
    # Calculate the proportion of each block group within the buffer zone
    bg_inbuffer['intersect_prop'] = bg_inbuffer['intersect_area'] / bg_inbuffer['shape_area_new']
    
    # Rename geometry col to intersect_geom so it's clear these geoms are just the intersections
    bg_inbuffer.rename(columns={'geometry':'intersect_geom'}, inplace=True)  # old:new. Match col names for merging

    # merge with ca_bg block groups to get full BG polygon geoms back in the df
    bg_inbuffer = pd.merge(bg_inbuffer, ca_blockgroups, on='ID')

    # Rename geometry column for clarity
    bg_inbuffer.rename(columns={'geometry':'bg_geom'}, inplace=True)  # old:new. Match col names for merging
    
    bg_inbuffer = bg_inbuffer.set_geometry('bg_geom')  # set to the buffers rather than the points

    # merge the intersected data back into the full df of all block groups
    df_final = gp.sjoin(social_df, bg_inbuffer, how = "left", predicate = 'contains')
    
    # Filter cols to just what's needed
    df_final = df_final[['ID_left', 'CNTY_NAME_left', 'ACSTOTPOP_left', 'MINORPOP_left', 'D_PM25_2_left', 'shape_area_new_left', 'intersect_area', 'intersect_prop', 'geometry', 'intersect_geom']]

    # clean things up after the sjoin
    new_cols = [col.replace('_left', '') if col.endswith('_left') else col for col in df_final.columns]
    df_final.columns = new_cols
    
    # Rename geometry column for clarity
    df_final.rename(columns={'geometry':'bg_geom'}, inplace=True)  # old:new. Match col names for merging

    # Replace NAs in both the intersect_prop and intersect_area with 0 for BG that have no intersection
    no_intersect = df_final['intersect_prop'].isna() | df_final['intersect_area'].isna()

    # Update the values in the 'intersect_prop' and 'intersect_area' columns of those rows to 0 using the loc method
    df_final.loc[no_intersect, ['intersect_prop', 'intersect_area']] = 0
    
    # Apply the proportion to each demographic variable to find counts by variable
    demo_vars = ['ACSTOTPOP', 'MINORPOP']
    for var in demo_vars:
        df_final[var + '_intersect_count'] = df_final[var] * df_final['intersect_prop']

    # find overall proportions for each demo var by dividing var count by respective total population    
    for var in demo_vars:
        df_final[var + '_bg_totprop'] = df_final[var] / df_final['ACSTOTPOP']
    
    # Finally, save a subsetted version of the df for permutation testing
    df_forpermutation = df_final[['MINORPOP_bg_totprop', 'intersect_prop']]
    df_forpermutation.to_csv(F"data/df_{buffer_size}m_forpermutation.csv", index=False)
    
    return df_final

In [ ]:
def calc_proportions(df, buffer_size, gen_table):
    
    df.to_csv("data/testing_calc.csv")
    
    in_buffer = np.sum(df['MINORPOP_bg_totprop'] * df['intersect_prop']) / np.sum(df['intersect_prop'])
    out_buffer = (np.sum(df['MINORPOP_bg_totprop'] * (1 - df['intersect_prop'])) /
                  np.sum(1 - df['intersect_prop']))
    
    # table that compares the in-buffer proportions to the outside-buffer proportions
    temp = pd.DataFrame({'ID': [f"buffer_{buffer_size}"], 'Buffer': [buffer_size], 
                         'in_buffer': [in_buffer], 'out_buffer': [out_buffer]})
    
    
    gen_table = pd.concat([gen_table, temp], ignore_index=True)
    
    return gen_table
    

In [ ]:
def calc_counts(df, buffer_size, gen_table):

    in_buffer = df['MINORPOP_intersect_count'].sum()
    out_buffer = df['ACSTOTPOP'].sum() - in_buffer

    # table that compares the in-buffer proportions to the outside-buffer proportions
    temp = pd.DataFrame({'ID': [f"buffer_{buffer_size}"], 'Buffer': [buffer_size], 'in_buffer': [in_buffer], 'out_buffer': [out_buffer]})
    
    gen_table = pd.concat([gen_table, temp], ignore_index=True)
    
    return gen_table

In [ ]:
def combined_calcs(flares_df, social_df, ca_blockgroups, buffer_size, gen_table):
    
    intersect = create_buffer_intersection(flares_df, social_df, buffer_size)
    df = calc_intersect_stats(intersect, social_df, ca_blockgroups, buffer_size)
    
    gen_table = calc_proportions(df, buffer_size, gen_table)

    return gen_table

In [ ]:
def all_buffer_calcs(gen_table, buffer_sizes = [100, 400, 800, 1000, 1600, 2000]):

    for buffer_size in buffer_sizes:
        gen_table = combined_calcs(ca_flares, ca_bg_joined, ca_bg, buffer_size, gen_table.copy())  
        
    return gen_table

### Proportions table

In [ ]:
# initialize empty df
prop_table = pd.DataFrame(columns=['ID', "Buffer", 'in_buffer', 'out_buffer'])
name = prop_table.name = 'Proportions Table'

In [ ]:
prop_table = all_buffer_calcs(prop_table, buffer_sizes=[100, 400])
prop_table['Buffer'] = prop_table['Buffer'].astype(float)

In [ ]:
prop_table

#### *The out-buffer proportions are not being calculated correctly. Code seems right, though. Troubleshoot with JC*

In [ ]:
# scatterplot of buffer props

fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(prop_table['Buffer'], prop_table['in_buffer'], label='Inside buffer')
ax.scatter(prop_table['Buffer'], prop_table['out_buffer'], label='Outside buffer')

# Set xticks to the buffer sizes
xticks = [100, 400, 800, 1000, 1600, 2000]
ax.set_xticks(xticks)

# Set xticklabels to the buffer sizes
xticklabels = [f"{b:,}" for b in xticks]
ax.set_xticklabels(xticklabels)
ax.set_xlabel('Buffer size (m)')
ax.set_ylabel('Proportion of MINORPOP')
ax.legend()
plt.show()


### Counts table

In [ ]:
# initialize empty df
count_table = pd.DataFrame(columns=['ID', "Buffer", 'in_buffer', 'out_buffer'])
name = count_table.name = 'Counts Table'

In [ ]:
# def combined_calcs(flares_df, social_df, ca_blockgroups, buffer_size, gen_table):
    
#     intersect = create_buffer_intersection(flares_df, social_df, buffer_size)
#     df = calc_intersect_stats(intersect, social_df, ca_blockgroups, buffer_size)
    
#     #gen_table = calc_proportions(df, buffer_size, gen_table)
#     gen_table = calc_counts(df, buffer_size, gen_table)

#     return gen_table

In [ ]:
count_table = all_buf_calcs(count_table)
count_table['Buffer'] = count_table['Buffer'].astype(float)

In [ ]:
count_table

In [ ]:
# scatterplot of buffer counts
# visualizing just in-buffer counts for readability. 

fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(count_table['Buffer'], count_table['in_buffer'], label='Inside buffer')
#ax.scatter(count_table['Buffer'], count_table['out_buffer'], label='Outside buffer')

# Define the formatter function
def thousand_fmt(x, pos):
    return f"{x:,.0f}"
# Set the formatter function for the y-axis
ax.yaxis.set_major_formatter(mtick.FuncFormatter(thousand_fmt))

# Set xticks to the buffer sizes
xticks = [100, 400, 800, 1000, 1600, 2000]
ax.set_xticks(xticks)

# Set xticklabels to the buffer sizes
xticklabels = [f"{b:,}" for b in xticks]
ax.set_xticklabels(xticklabels)
ax.set_xlabel('Buffer size (m)')
ax.set_ylabel('Counts of MINORPOP')
ax.legend()
plt.show()

## Visualization with Folium

In [ ]:
# Convert the GeoDataFrame to the same CRS as the folium map (if necessary)

# Create a folium map with a center location
m = folium.Map(location=[38.377158,-121.645792], zoom_start=6, tiles=None,overlay=False)  #start w lat/long roughly in center of CA
base_map = folium.FeatureGroup(name='Base map', overlay=True, control=False)
folium.TileLayer(tiles='OpenStreetMap').add_to(base_map)
base_map.add_to(m)

In [ ]:
# # Choropleth of Block Groups

# folium.Choropleth(intersect_df,
#                   data=intersect_df,
#                   columns = ['bg_id', 'D_PM25_2'], 
#                   key_on='feature.properties.bg_id',
#                   fill_color="Reds",
#                   fill_opacity=0.7,
#                   line_opacity=0.2,
#                   legend_name="PM2.5 Index",
#                  name="Block Groups").add_to(m)


In [ ]:
# # Feature Group: All Block Groups

# def style_function3(feature):
#     return {
#         'fillColor': 'grey',
#         'color': 'grey',
#         'fillOpacity': 0.05
#     }

# all_bg = folium.FeatureGroup(name='All BG', overlay=True)
# folium.GeoJson(
#     data=ca_bg["geometry"],
#     style_function=style_function3
# ).add_to(all_bg)
# all_bg.add_to(m)

In [ ]:
# Feature Group: Buffers

# def style_function1(feature):
#     return {
#         'fillColor': 'red',
#         'color': 'red',
#         'fillOpacity': 0.2
#     }

# all_flares_buffers = folium.FeatureGroup(name='Flare Buffers', overlay=True)
# folium.GeoJson(
#     data=ca_flares["buffer_2000m"],
#     style_function=style_function1
# ).add_to(all_flares_buffers)
# all_flares_buffers.add_to(m)


# Feature Group: Buffers

def style_function1(feature):
    return {
        'fillColor': 'red',
        'color': 'red',
        'fillOpacity': 0.2
    }

all_flares_buffers = folium.FeatureGroup(name='Flare Buffers', overlay=True)
folium.GeoJson(
    data=intersect_df["cnty_geom"],
    style_function=style_function1
).add_to(all_flares_buffers)
all_flares_buffers.add_to(m)

In [ ]:
# Feature Group: BG-Buffer Intersections

def style_function2(feature):
    return {
        'fillColor': 'blue',
        'color': 'blue',
        'fillOpacity': 0.1
    }

intersect = folium.FeatureGroup(name='BG-Flare Intersections', overlay=True)
folium.GeoJson(
    data=intersect_df['intersect_geom'],
    style_function=style_function2
).add_to(intersect)
intersect.add_to(m)

In [ ]:
# # Add hover functionality
# test = test.to_crs("EPSG:3857")
# test = test.set_geometry('bg_geom')
# test_json = test.__geo_interface__

In [ ]:

# style_function = lambda x: {'fillColor': '#ffffff', 
#                             'color':'#000000', 
#                             'fillOpacity': 0.1, 
#                             'weight': 0.1}
# highlight_function = lambda x: {'fillColor': '#999999', 
#                                 'color':'#999999', 
#                                 'fillOpacity': 0.50, 
#                                 'weight': 0.1}
# NIL = folium.features.GeoJson(
#     data = map_test,
#     style_function=style_function, 
#     control=False,
#     highlight_function=highlight_function, 
#     tooltip=folium.features.GeoJsonTooltip(
#         fields=['ID','CNTY_NAME'],# 'D_PM25_2', 'ACSTOTPOP', 'MINORPOP','shape_area_new', 'intersect_prop', 'intersect_area', 'MINORPOP_bg_totprop'],
#         style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
#     )
# )

In [ ]:
# # add hover functionality as child to map, add layering, display map
# m.add_child(NIL)
# m.keep_in_front(NIL)
# folium.LayerControl().add_to(m)
# m

In [ ]:
# Flares
all_flares_points = folium.FeatureGroup(name='flare points', overlay=True)
folium.GeoJson(data=ca_flares["geometry"]).add_to(all_flares_points)
all_flares_points.add_to(m)

In [ ]:
m